In [1]:
#-------------------------------------------------------------------------------------------------------------------------------
# By Alexandra Lee (July 2018) 
#
# Apply saved model to new samples including:
#
# Encode samples from new condition using saved model
# Encode test set using saved model
# Decode estimated gene experssion after LSA
#-------------------------------------------------------------------------------------------------------------------------------
import os
import pandas as pd
import numpy as np
from keras.models import model_from_json, load_model
from keras import metrics, optimizers
np.random.seed(123)

Using TensorFlow backend.


In [2]:
# load arguments
input_file = os.path.join(os.path.dirname(os.getcwd()), "encoded", "tybalt_2layer_10_test_control_encoded.txt")
model_file = os.path.join(os.path.dirname(os.getcwd()), "models", "tybalt_2layer_10_train_decoder_model.h5")
weights_file = os.path.join(os.path.dirname(os.getcwd()), "models", "tybalt_2layer_10_train_decoder_weights.h5")

# If encoding
encoding = True

# output files
out_file = os.path.join(os.path.dirname(os.getcwd()), "output", "estimated_test_control_latent_2layer.txt")


In [3]:
# read in data
data = pd.read_table(input_file, header = 0, sep = '\t', index_col = 0)
data

,0,1,2,3,4,5,6,7,8,9
GSM356955.CEL,1.557081,0.000000,0.000000,0.000000,4.812786,0.000000,0.296684,0.318213,4.430712,0.455239
GSM954576_Nomura_PA01-1_Pae_G1a_.CEL,0.405962,0.000000,0.749353,5.579066,3.664676,4.965723,0.000000,1.020203,3.017876,0.000000
GSM954578_Nomura_PA01-3_Pae_G1a_.CEL,0.245053,0.000000,0.261679,7.001842,1.163499,5.724838,0.000000,2.427797,2.109365,0.000000
GSM954579_Nomura_PA01-4_Pae_G1a_.CEL,0.361329,0.080056,0.176546,6.583868,1.589735,5.530672,0.000000,2.284974,2.120986,0.000000
GSM92182.CEL,4.610730,0.391221,0.050018,6.894863,0.194969,2.032050,2.152950,2.695515,0.000000,3.091622


In [4]:
# read in saved models

# load json and create model
#json_file = open(model_file, 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#loaded_model = model_from_json(loaded_model_json)
   
loaded_model = load_model(model_file)
# load weights into new model
loaded_model.load_weights(weights_file)

/home/alexandra/anaconda3/envs/Pa/lib/python3.5/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
# Use trained model to encode new data into SAME latent space
reconstructed = loaded_model.predict_on_batch(data)

if encoding:
    reconstructed_df = pd.DataFrame(reconstructed, index=data.index)
else:
    reconstructed_df = pd.DataFrame(reconstructed) # Can we assume the index is preserved after decoding?

reconstructed_df

,0,1,2,3,4,5,6,7,8,9,...,5539,5540,5541,5542,5543,5544,5545,5546,5547,5548
GSM356955.CEL,0.477484,0.515526,0.363541,0.618800,0.365553,0.411387,0.427771,0.563649,0.520676,0.244805,...,0.438112,0.579595,0.480473,0.689857,0.623972,0.579436,0.688972,0.750287,0.561240,0.449993
GSM954576_Nomura_PA01-1_Pae_G1a_.CEL,0.521049,0.619420,0.347263,0.559277,0.315040,0.378935,0.449362,0.484582,0.405961,0.190307,...,0.385676,0.540066,0.483354,0.542590,0.571044,0.275210,0.436921,0.416739,0.491154,0.510596
GSM954578_Nomura_PA01-3_Pae_G1a_.CEL,0.605867,0.672940,0.356914,0.566674,0.362107,0.349446,0.496895,0.460210,0.405965,0.153048,...,0.420379,0.550845,0.582028,0.533298,0.617527,0.161721,0.348830,0.395203,0.568011,0.645493
GSM954579_Nomura_PA01-4_Pae_G1a_.CEL,0.603720,0.673259,0.360111,0.567868,0.360127,0.354570,0.491138,0.464283,0.410305,0.157080,...,0.417571,0.554011,0.577131,0.543280,0.622229,0.174536,0.361174,0.404193,0.572957,0.640457
GSM92182.CEL,0.679434,0.731911,0.488280,0.713364,0.461989,0.379543,0.423460,0.538023,0.577771,0.165634,...,0.316400,0.544758,0.587342,0.613760,0.642981,0.124814,0.398410,0.525236,0.696359,0.657144


In [6]:
# Save latent space representation
reconstructed_df.to_csv(out_file, sep='\t')